In [1]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performerErrorTypeTest_V2 as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

nltk.download('punkt')
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import glob

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import csv
def readCSV(file_name):
    errlist=[]
    LBlist=[]
    with open(file_name, newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary

        rows = csv.DictReader(csvfile)
        for row in rows: 
            RL=list(row.values())
            #print("RL[0]: ", type(RL[0]), "RL[1]: ", type(RL[1]))
            RL[1:] = list(map(int, RL[1:]))
            errs=RL[1:37]
            LB=RL[37:]
            errlist.append(errs)
            LBlist.append(LB)
    return errlist,LBlist


In [3]:
def loadTestTrainData(filename): # e.g., 'test.npy'
    with open(filename, "rb") as f:
        a = np.load(f)
        return a

In [4]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [5]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [6]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'NUM_INT', '>'], ["<NUM_INT>"])
    replace_sublist(x, ['<', 'NUM_FLOAT', '>'], ["<NUM_FLOAT>"])
    replace_sublist(x, ['<', 'STRING', '>'], ["<STRING>"])
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [7]:
def parseSentence(x):	
    tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        #print(ord(x[i]))
        if (ord(x[i])>255):
            inp="U"
        else:
            inp="E"

        if state=="START":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])

        elif state=="ASCII":
            if inp=="E":
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="UNICODE":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])
    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
    return replaceTAGS(tokens)

In [8]:
def readcode(fname):
    with open(fname,encoding = 'utf-8') as f:
        data = f.read()
        return data

In [9]:
import os
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [10]:
#save model for training
class TestTranslate(unittest.TestCase):
        
    def __init__(self):
        self.source_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<CR>': 5,
            '<NUM_INT>': 6,
            '<NUM_FLOAT>': 7,
            '<STRING>': 8,
        }
        
    @staticmethod
    def _build_token_dict(token_dict, token_list):
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
    
    def test_translate(self):
        #print("i am here: " )
        #source_file=[]
        #Set Para
        max_javaline_length = 160 #Max number of lines
        #set path
        Output_Path = "Trianing\InputCSV\Split-500-reduce-binary-accuracy"
        Input_Path = "Trianing\InputTxt\Split-500-reduce-binary-accuracy"
        model_for_training_org_path = "Model-for-training-org\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        model_for_training_path = "Model-for-training\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000"
        Trained_model_Path = "Trained_models\ErrorType-Test\Split-500-reduce-binary-accuracy\Max-len-1000\Learning-rate-3x0\weight-1vs1-V2"
        source_max_len_name = "source_max_len"
        type_weight = 1
        line_weight = 1
        learning_rate_value = 0.0001
        #get all txt file in input path
        target_max_len = 0
        token_num = 0

        #start training
        from Data_buffer import DataGeneratorTrainErrorTypeEdition as DGTrain
        from Data_buffer import DataGeneratorValidationErrorTypeEdition as DGValidation
        from Data_buffer import DataBuffer as db
        from random import randrange
        source_token_dict_name = "source_token_dict.pickle"
        #load source_token_dict
        source_token_dict = loadDictionary(model_for_training_path + "/" + source_token_dict_name)
        #load source_max_len
        source_max_len = loadDictionary(Trained_model_Path + "/" + source_max_len_name)
        #load training models and their len
        x_train = loadTestTrainData(model_for_training_path + "/" + "x_train_0.npy")
        x_validation = loadTestTrainData(model_for_training_path + "/" + "x_validation_0.npy")
        
        buffer_train_num = len(x_train)
        buffer_val_train_num = len(x_validation)
        #Set model para    
        model = tfr.get_model(max_input_len=(source_max_len),
                              errNum=36,
                              token_num=len(source_token_dict),
                              embed_dim=32, #32, try 32 or 64
                              encoder_num=6, #2 max = 6
                              head_num=4,#4
                              hidden_dim=128, #128
                              dropout_rate=0.05 #0.05
                             )
        #Set losses
        losses = {"error_feed_forward_output1": "binary_crossentropy"}
        #error type weight
        lossWeights = {"error_feed_forward_output1": type_weight}
        metrics = {"error_feed_forward_output1": "binary_accuracy"}
        #metrics = {"error_feed_forward_output1": tf.keras.metrics.Accuracy()}
        
        #set complie para
        model.compile(optimizer=Adam(learning_rate=learning_rate_value), loss=losses, loss_weights=lossWeights, metrics=metrics)
        
        
        #for output
        #for x
        #data_number and block_size = data num
        input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": buffer_train_num,
            "data_type": int,
            "block_size": buffer_train_num 
            }
        
        #for input
        #for y
        #data_number and block_size = data num
        """
        output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [buffer_train_num, buffer_train_num],
            "data_type": [int, int],
            "block_size": [buffer_train_num, buffer_train_num] 
            }
        """
        output_buffer_params = {
            "data_path": [model_for_training_path],
            "data_number": [buffer_train_num],
            "data_type": [int],
            "block_size": [buffer_train_num] 
            }
        
        #===========================================
        #for output
        #for x
        #data_number and block_size = validation data num
        
        validation_input_buffer_params = { 
            "data_path": model_for_training_path,
            "data_number": buffer_val_train_num,
            "data_type": int,
            "block_size": buffer_val_train_num 
            }
        
        #for input
        #for y
        #data_number and block_size = validation data num
        """
        validation_output_buffer_params = {
            "data_path": [model_for_training_path, model_for_training_path],
            "data_number": [buffer_val_train_num, buffer_val_train_num],
            "data_type": [int, int],
            "block_size": [buffer_val_train_num, buffer_val_train_num] 
            }
        """
        
        validation_output_buffer_params = {
            "data_path": [model_for_training_path],
            "data_number": [buffer_val_train_num],
            "data_type": [int],
            "block_size": [buffer_val_train_num] 
            }
        
        
        #Create Generators
        print("Creating training generator...")
        #give training data num
        """training_generator = DGTrain.DataGeneratorTrain(input_buffer_params,
                                                           output_buffer_params,
                                                           [list(range(buffer_val_train_num)), list(range(buffer_val_train_num))] 
                                                )"""
        training_generator = DGTrain.DataGeneratorTrainErrorTypeEdition(input_buffer_params,
                                                                        output_buffer_params,
                                                                        [list(range(buffer_val_train_num))] 
                                                )
        #Create Generators
        print("Creating validation generator...")
        #give valitdation data num
        """validation_generator = DGValidation.DataGeneratorValidation(validation_input_buffer_params,
                                                                       validation_output_buffer_params,
                                                                      [list(range(buffer_val_train_num)), list(range(buffer_val_train_num))] 
                                                )"""
        validation_generator = DGValidation.DataGeneratorValidationErrorTypeEdition(validation_input_buffer_params,
                                                  validation_output_buffer_params,
                                                  [list(range(buffer_val_train_num))] 
                                                )
        
        
        #''' <-----traing switch
        #Start training
        print("Strat training...")
        
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = Trained_model_Path + "/" + "checkpoint_model.h5",
                                                                       monitor = "val_loss",
                                                                       mode = "min",
                                                                       save_best_only = True
                                                                      )
        
        history = model.fit_generator(generator = training_generator,
                                      epochs = 1000, #100 200 500 3000
                                      verbose = 2, #set visibility
                                      validation_data = validation_generator,
                                      callbacks = [model_checkpoint_callback],
                                     )
        
        
        print("Model training completed...")
        #save history
        print("Saving history...")
        saveDictionary(history.history, Trained_model_Path + "/" + "model_history")
        print("History saving completed...")
        
        #save model
        print("Saving model...")
        model.save(Trained_model_Path + "/" + "test_model1.h5")
        print("Model saving completed...")
        
        #print("history.history.keys: ", history.history.keys())\
        #'''
                
    def getsource_max_lan(self):
        return self.sl
    

In [11]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [12]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [13]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [14]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [15]:
x=TestTranslate()
x.test_translate()

Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Creating training generator...
Creating validation generator...
Strat training...


C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/1000
22/22 - 30s - loss: 0.2687 - binary_accuracy: 0.9275 - val_loss: 0.1996 - val_binary_accuracy: 0.9536
Epoch 2/1000


C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


22/22 - 17s - loss: 0.1926 - binary_accuracy: 0.9540 - val_loss: 0.1897 - val_binary_accuracy: 0.9537
Epoch 3/1000
22/22 - 17s - loss: 0.1892 - binary_accuracy: 0.9538 - val_loss: 0.1915 - val_binary_accuracy: 0.9536
Epoch 4/1000
22/22 - 12s - loss: 0.1892 - binary_accuracy: 0.9535 - val_loss: 0.1888 - val_binary_accuracy: 0.9540
Epoch 5/1000
22/22 - 12s - loss: 0.1869 - binary_accuracy: 0.9537 - val_loss: 0.1892 - val_binary_accuracy: 0.9536
Epoch 6/1000
22/22 - 17s - loss: 0.1856 - binary_accuracy: 0.9537 - val_loss: 0.1841 - val_binary_accuracy: 0.9538
Epoch 7/1000
22/22 - 12s - loss: 0.1849 - binary_accuracy: 0.9538 - val_loss: 0.1875 - val_binary_accuracy: 0.9537
Epoch 8/1000
22/22 - 12s - loss: 0.1858 - binary_accuracy: 0.9538 - val_loss: 0.1856 - val_binary_accuracy: 0.9534
Epoch 9/1000
22/22 - 17s - loss: 0.1835 - binary_accuracy: 0.9541 - val_loss: 0.1848 - val_binary_accuracy: 0.9536
Epoch 10/1000
22/22 - 12s - loss: 0.1842 - binary_accuracy: 0.9540 - val_loss: 0.1845 - val_b

...

Epoch 700/1000
22/22 - 12s - loss: 3.0820e-04 - binary_accuracy: 0.9999 - val_loss: 0.4527 - val_binary_accuracy: 0.9582
Epoch 701/1000
22/22 - 17s - loss: 2.6322e-04 - binary_accuracy: 0.9999 - val_loss: 0.4508 - val_binary_accuracy: 0.9582
Epoch 702/1000
22/22 - 17s - loss: 2.9552e-04 - binary_accuracy: 0.9999 - val_loss: 0.4532 - val_binary_accuracy: 0.9582
Epoch 703/1000
22/22 - 12s - loss: 3.5926e-04 - binary_accuracy: 0.9999 - val_loss: 0.4527 - val_binary_accuracy: 0.9584
Epoch 704/1000
22/22 - 17s - loss: 2.5836e-04 - binary_accuracy: 1.0000 - val_loss: 0.4540 - val_binary_accuracy: 0.9586
Epoch 705/1000
22/22 - 12s - loss: 2.9264e-04 - binary_accuracy: 0.9999 - val_loss: 0.4551 - val_binary_accuracy: 0.9585
Epoch 706/1000
22/22 - 17s - loss: 3.3564e-04 - binary_accuracy: 0.9999 - val_loss: 0.4586 - val_binary_accuracy: 0.9579
Epoch 707/1000
22/22 - 12s - loss: 2.7335e-04 - binary_accuracy: 0.9999 - val_loss: 0.4576 - val_binary_accuracy: 0.9584
Epoch 708/1000
22/22 - 12s - los

Epoch 768/1000
22/22 - 18s - loss: 3.0335e-04 - binary_accuracy: 0.9999 - val_loss: 0.4853 - val_binary_accuracy: 0.9578
Epoch 769/1000
22/22 - 18s - loss: 2.4627e-04 - binary_accuracy: 0.9999 - val_loss: 0.4875 - val_binary_accuracy: 0.9576
Epoch 770/1000
22/22 - 18s - loss: 2.1294e-04 - binary_accuracy: 0.9999 - val_loss: 0.4817 - val_binary_accuracy: 0.9581
Epoch 771/1000
22/22 - 18s - loss: 2.3434e-04 - binary_accuracy: 0.9999 - val_loss: 0.4891 - val_binary_accuracy: 0.9577
Epoch 772/1000
22/22 - 18s - loss: 3.3328e-04 - binary_accuracy: 0.9999 - val_loss: 0.4884 - val_binary_accuracy: 0.9583
Epoch 773/1000
22/22 - 18s - loss: 2.3066e-04 - binary_accuracy: 0.9999 - val_loss: 0.4930 - val_binary_accuracy: 0.9580
Epoch 774/1000
22/22 - 18s - loss: 2.1859e-04 - binary_accuracy: 0.9999 - val_loss: 0.4883 - val_binary_accuracy: 0.9577
Epoch 775/1000
22/22 - 17s - loss: 2.6004e-04 - binary_accuracy: 0.9999 - val_loss: 0.4888 - val_binary_accuracy: 0.9584
Epoch 776/1000
22/22 - 18s - los

Epoch 836/1000
22/22 - 18s - loss: 2.9666e-04 - binary_accuracy: 0.9999 - val_loss: 0.5054 - val_binary_accuracy: 0.9575
Epoch 837/1000
22/22 - 13s - loss: 2.9131e-04 - binary_accuracy: 0.9999 - val_loss: 0.5019 - val_binary_accuracy: 0.9581
Epoch 838/1000
22/22 - 17s - loss: 3.4273e-04 - binary_accuracy: 0.9999 - val_loss: 0.4974 - val_binary_accuracy: 0.9580
Epoch 839/1000
22/22 - 18s - loss: 3.3544e-04 - binary_accuracy: 0.9999 - val_loss: 0.4984 - val_binary_accuracy: 0.9581
Epoch 840/1000
22/22 - 18s - loss: 2.2781e-04 - binary_accuracy: 0.9999 - val_loss: 0.4994 - val_binary_accuracy: 0.9580
Epoch 841/1000
22/22 - 18s - loss: 2.5785e-04 - binary_accuracy: 0.9999 - val_loss: 0.5001 - val_binary_accuracy: 0.9578
Epoch 842/1000
22/22 - 18s - loss: 3.2061e-04 - binary_accuracy: 0.9999 - val_loss: 0.5059 - val_binary_accuracy: 0.9584
Epoch 843/1000
22/22 - 18s - loss: 1.7066e-04 - binary_accuracy: 0.9999 - val_loss: 0.5053 - val_binary_accuracy: 0.9584
Epoch 844/1000
22/22 - 18s - los

Epoch 904/1000
22/22 - 18s - loss: 2.1186e-04 - binary_accuracy: 0.9999 - val_loss: 0.5150 - val_binary_accuracy: 0.9584
Epoch 905/1000
22/22 - 13s - loss: 2.5944e-04 - binary_accuracy: 0.9999 - val_loss: 0.5158 - val_binary_accuracy: 0.9583
Epoch 906/1000
22/22 - 18s - loss: 1.8767e-04 - binary_accuracy: 0.9999 - val_loss: 0.5113 - val_binary_accuracy: 0.9580
Epoch 907/1000
22/22 - 18s - loss: 1.8802e-04 - binary_accuracy: 0.9999 - val_loss: 0.5191 - val_binary_accuracy: 0.9581
Epoch 908/1000
22/22 - 18s - loss: 2.5271e-04 - binary_accuracy: 0.9999 - val_loss: 0.5204 - val_binary_accuracy: 0.9580
Epoch 909/1000
22/22 - 18s - loss: 1.8725e-04 - binary_accuracy: 0.9999 - val_loss: 0.5215 - val_binary_accuracy: 0.9582
Epoch 910/1000
22/22 - 18s - loss: 2.6097e-04 - binary_accuracy: 0.9999 - val_loss: 0.5209 - val_binary_accuracy: 0.9578
Epoch 911/1000
22/22 - 17s - loss: 2.7230e-04 - binary_accuracy: 0.9999 - val_loss: 0.5180 - val_binary_accuracy: 0.9579
Epoch 912/1000
22/22 - 18s - los

Epoch 972/1000
22/22 - 17s - loss: 2.8814e-04 - binary_accuracy: 0.9999 - val_loss: 0.5335 - val_binary_accuracy: 0.9573
Epoch 973/1000
22/22 - 17s - loss: 1.7129e-04 - binary_accuracy: 0.9999 - val_loss: 0.5335 - val_binary_accuracy: 0.9574
Epoch 974/1000
22/22 - 12s - loss: 2.8141e-04 - binary_accuracy: 0.9999 - val_loss: 0.5408 - val_binary_accuracy: 0.9574
Epoch 975/1000
22/22 - 17s - loss: 4.0286e-04 - binary_accuracy: 0.9998 - val_loss: 0.5354 - val_binary_accuracy: 0.9565
Epoch 976/1000
22/22 - 17s - loss: 5.4171e-04 - binary_accuracy: 0.9998 - val_loss: 0.5415 - val_binary_accuracy: 0.9559
Epoch 977/1000
22/22 - 12s - loss: 0.0013 - binary_accuracy: 0.9997 - val_loss: 0.5386 - val_binary_accuracy: 0.9567
Epoch 978/1000
22/22 - 17s - loss: 0.0016 - binary_accuracy: 0.9995 - val_loss: 0.5529 - val_binary_accuracy: 0.9549
Epoch 979/1000
22/22 - 17s - loss: 0.0029 - binary_accuracy: 0.9991 - val_loss: 0.5411 - val_binary_accuracy: 0.9562
Epoch 980/1000
22/22 - 12s - loss: 0.0025 - 